In [84]:
from transformers import AutoTokenizer, TFAutoModel,TFAutoModelForSequenceClassification
import tensorflow as tf

import pandas as pd
import numpy as np

In [73]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [44]:
email_data= pd.read_csv("email_classification.csv")
email_data.head()
print(email_data.shape)

(179, 2)


In [66]:
# Load pre-trained model and tokenizer
model_name = "distilbert-base-uncased"  # e.g., "bert-base-uncased", "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

# List of sentences/emails
emails = email_data["email"].tolist()

# Tokenize and encode emails
encoded_inputs = tokenizer(emails, padding=True, truncation=True, return_tensors="tf")

# Forward pass
outputs = model(encoded_inputs)

# Extract the output embeddings (CLS token for BERT-based models)
# For BERT-based models, we take the output from the last hidden layer and the first token [CLS]
# For other models, you might need to adjust this
embeddings = outputs.last_hidden_state[:, 0, :].numpy()

print("Shape of embeddings:", embeddings.shape)
# Now, 'embeddings' contains the vectors for your emails


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Shape of embeddings: (179, 768)


In [67]:
label_mapping = {"ham": 0, "spam": 1}

# Map the labels using the mapping
y = email_data["label"].map(label_mapping)

In [68]:
X=embeddings

In [69]:
print(X.shape)
y.shape

(179, 768)


(179,)

In [70]:
# Shuffle the indices
indices = np.arange(len(X))
np.random.shuffle(indices)

# Define the split ratio
train_ratio = 0.8  # 80% train, 20% test

# Calculate split indices
split_idx = int(len(X) * train_ratio)
train_indices = indices[:split_idx]
test_indices = indices[split_idx:]

# Split the data
X_train, X_test = tf.gather(X, train_indices), tf.gather(X, test_indices)

# Assuming you have labels y, you can split them as well
y_train, y_test = y[train_indices], y[test_indices]

# Now you have X_train, y_train, X_test, y_test for training and testing

In [71]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(143, 768)
(36, 768)
(143,)
(36,)


In [83]:


# Create a KNN classifier
k = 1  # Number of neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Train the classifier
knn_classifier.fit(X_train, y_train)

# Make predictions
predictions = knn_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9444444444444444
